## Abstract

The objective of this case study is to check for data quality issues.

A little about data:
1. receipts.xlsx
2. users.xlsx
3. brands.xlsx

#### receipts.xlsx:
* _id: uuid for this receipt
* bonusPointsEarned: Number of bonus points that were awarded upon receipt completion
* bonusPointsEarnedReason: event that triggered bonus points
* createDate: The date that the event was created
* dateScanned: Date that the user scanned their receipt
* finishedDate: Date that the receipt finished processing
* modifyDate: The date the event was modified
* pointsAwardedDate: The date we awarded points for the transaction
* pointsEarned: The number of points earned for the receipt
* purchaseDate: the date of the purchase
* purchasedItemCount: Count of number of items on the receipt
* rewardsReceiptItemList: The items that were purchased on the receipt
* rewardsReceiptStatus: status of the receipt through receipt validation and processing
* totalSpent: The total amount on the receipt
* userId: string id back to the User collection for the user who scanned the receipt

#### users.xlsx:
* _id: user Id
* state: state abbreviation
* createdDate: when the user created their account
* lastLogin: last time the user was recorded logging in to the app
* role: constant value set to 'CONSUMER'
* active: indicates if the user is active; only Fetch will de-activate an account with this flag

#### brands.xlsx
* _id: brand uuid
* barcode: the barcode on the item
* brandCode: String that corresponds with the brand column in a partner product file
* category: The category name for which the brand sells products in
* categoryCode: The category code that references a BrandCategory
* cpg: reference to CPG collection
* topBrand: Boolean indicator for whether the brand should be featured as a 'top brand'
* name: Brand name


In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt  # Library for visualisation
import numpy as np               # Numeric library for calculations
import pandas as pd              # Library for data analysis
from scipy import stats          # Library for advanced scientific calculations
import seaborn as sns            # Library for statistical visualization
import re

In [2]:
## reading the dataset
df_receipts = pd.read_excel("receipts.xlsx")
df_users = pd.read_excel("users.xlsx")
df_brands = pd.read_excel('brands.xlsx')

In [4]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,$oid:5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",$date:1609687531000,$date:1609687531000,$date:1609687531000,$date:1609687536000,$date:1609687531000,500.0,$date:1609632000000,5.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,$oid:5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",$date:1609687483000,$date:1609687483000,$date:1609687483000,$date:1609687488000,$date:1609687483000,150.0,$date:1609601083000,2.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,$oid:5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,$date:1609687537000,$date:1609687537000,NaN,$date:1609687542000,NaN,5.0,$date:1609632000000,1.0,needsFetchReview:false|partnerItemId:1|prevent...,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,$oid:5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,$date:1609687534000,$date:1609687534000,$date:1609687534000,$date:1609687539000,$date:1609687534000,5.0,$date:1609632000000,4.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,$oid:5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,$date:1609687506000,$date:1609687506000,$date:1609687511000,$date:1609687511000,$date:1609687506000,5.0,$date:1609601106000,2.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [5]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
1,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
2,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
3,$oid:5ff1e1eacfcf6c399c274ae6,1.0,$date:1609687530554,$date:1609687530597,consumer,Email,WI
4,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI


In [6]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,$oid:601ac115be37ce2ead437551,511111019862,Baking,BAKING,$id:$oid:601ac114be37ce2ead437550|$ref:Cogs,test brand @1612366101024,NaN,NaN
1,$oid:601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,$id:$oid:5332f5fbe4b03c9a25efd0ba|$ref:Cogs,Starbucks,NaN,STARBUCKS
2,$oid:601ac142be37ce2ead43755d,511111819905,Baking,BAKING,$id:$oid:601ac142be37ce2ead437559|$ref:Cogs,test brand @1612366146176,NaN,TEST BRANDCODE @1612366146176
3,$oid:601ac142be37ce2ead43755a,511111519874,Baking,BAKING,$id:$oid:601ac142be37ce2ead437559|$ref:Cogs,test brand @1612366146051,NaN,TEST BRANDCODE @1612366146051
4,$oid:601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,$id:$oid:5332fa12e4b03c9a25efd1e7|$ref:Cogs,test brand @1612366146827,NaN,TEST BRANDCODE @1612366146827


#### Checking the concise summary of dataframes

In [7]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1102 non-null   object 
 13  totalSpent               667 non-null    float64
 14  userId                  

In [8]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           495 non-null    object 
 1   active        494 non-null    float64
 2   createdDate   495 non-null    object 
 3   lastLogin     433 non-null    object 
 4   role          495 non-null    object 
 5   signUpSource  447 non-null    object 
 6   state         439 non-null    object 
dtypes: float64(1), object(6)
memory usage: 27.2+ KB


In [9]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      31 non-null     float64
 7   brandCode     898 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


#### Checking for number of rows and columns in each dataset

In [10]:
df_receipts.shape

(1119, 15)

In [11]:
df_users.shape

(495, 7)

In [12]:
df_brands.shape

(1167, 8)

#### Checking for null in each dataset

In [14]:
df_receipts.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus        17
totalSpent                 452
userId                      17
dtype: int64

In [15]:
df_users.isnull().sum()

_id              0
active           1
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [16]:
df_brands.isnull().sum()

_id                0
barcode            0
category         155
categoryCode     650
cpg                0
name               0
topBrand        1136
brandCode        269
dtype: int64

#### From the above result, it can be inferred that all 3 dataset have null values. Brands and Receipts dataset have more number of null's than Users dataset. 

#### Handling null values :
* Drop null values
* Replace/Impute null values

#### If the fields containing null value are not significantly impacting the analysis,  then those columns can be dropped. Otherwise, the null values should be replaced with subsitute value depending on the type of data.

### Checking for duplicates

In [17]:
df_receipts.duplicated().sum()

0

In [19]:
df_users.duplicated().sum()

283

In [21]:
df_brands.duplicated().sum()

0

#### Only Users dataset has duplicate data. Duplicates can be dropped. 

### Some of the data quality issues with the given dataset are:

* Null
* Duplicates